In [30]:
import pandas as pd
import re

In [31]:
data = pd.read_csv('car_prices.csv')

In [32]:
data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST)
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)


In [33]:
data.dtypes

year              int64
make             object
model            object
trim             object
body             object
transmission     object
vin              object
state            object
condition       float64
odometer        float64
color            object
interior         object
seller           object
mmr             float64
sellingprice    float64
saledate         object
dtype: object

In [34]:
pattern = r'\b(\d{4})\b'
years = data['saledate'].str.extract(pattern)
len(years)


558837

In [35]:
pattern = r'[A-Z][a-z]{2}\s+([A-Z][a-z]{2})'
months = data['saledate'].str.extract(pattern)
len(months)

558837

In [36]:
data['sales_year'] = years
data['sales_month'] = months
data.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate,sales_year,sales_month
0,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg566472,ca,5.0,16639.0,white,black,kia motors america inc,20500.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),2014,Dec
1,2015,Kia,Sorento,LX,SUV,automatic,5xyktca69fg561319,ca,5.0,9393.0,white,beige,kia motors america inc,20800.0,21500.0,Tue Dec 16 2014 12:30:00 GMT-0800 (PST),2014,Dec
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST),2015,Jan
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST),2015,Jan
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST),2014,Dec


In [37]:
data = data.drop(['vin','trim','seller','saledate'], axis=1)

In [38]:
data.dropna()

,year,make,model,body,transmission,state,condition,odometer,color,interior,mmr,sellingprice,sales_year,sales_month
0,2015,Kia,Sorento,SUV,automatic,ca,5.0,16639.0,white,black,20500.0,21500.0,2014,Dec
1,2015,Kia,Sorento,SUV,automatic,ca,5.0,9393.0,white,beige,20800.0,21500.0,2014,Dec
2,2014,BMW,3 Series,Sedan,automatic,ca,45.0,1331.0,gray,black,31900.0,30000.0,2015,Jan
3,2015,Volvo,S60,Sedan,automatic,ca,41.0,14282.0,white,black,27500.0,27750.0,2015,Jan
4,2014,BMW,6 Series Gran Coupe,Sedan,automatic,ca,43.0,2641.0,gray,black,66000.0,67000.0,2014,Dec
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558831,2011,BMW,5 Series,Sedan,automatic,fl,39.0,66403.0,white,brown,20300.0,22800.0,2015,Jul
558833,2012,Ram,2500,Crew Cab,automatic,wa,5.0,54393.0,white,black,30200.0,30800.0,2015,Jul
558834,2012,BMW,X5,SUV,automatic,ca,48.0,50561.0,black,black,29800.0,34000.0,2015,Jul
558835,2015,Nissan,Altima,sedan,automatic,ga,38.0,16658.0,white,black,15100.0,11100.0,2015,Jul


In [39]:
map = {'Dec':'12', 'Jan':'1', 'Jul':'7', 'Feb':'2', 'Jun':'6', 'May':'5', 'Mar':'3', 'Apr':'4'}
data['sales_month'] = data['sales_month'].map(map)

In [40]:
data['sales_month'].value_counts()

sales_month
2     163053
1     140815
6      99937
12     53520
5      52447
3      46277
4       1450
7       1300
Name: count, dtype: int64

In [41]:
data.to_csv('car_prices_first_cleaning.csv')